In [3]:
import numpy as np
from sklearn import preprocessing
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter=',')
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

# Balance the Dataset

In [4]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []
for i in range(len(targets_all)):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_prior = np.delete(targets_all, indices_to_remove, axis=0)

# Standardize the inputs

In [5]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

# Shuffle the data

In [6]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_prior[shuffled_indices]

# Split the dataset

In [9]:
sample_count = shuffled_inputs.shape[0]
train_sample_count = int(0.8*sample_count)
validation_sample_count = int(0.1*sample_count)
test_sample_count = sample_count - train_sample_count - validation_sample_count

train_inputs = shuffled_inputs[:train_sample_count]
train_targets = shuffled_targets[:train_sample_count]

validation_inputs = shuffled_inputs[train_sample_count:train_sample_count+validation_sample_count]
validation_targets = shuffled_targets[train_sample_count:train_sample_count+validation_sample_count]

test_input = shuffled_inputs[train_sample_count+validation_sample_count:]
test_targets = shuffled_targets[train_sample_count+validation_sample_count:]

print(np.sum(train_targets), train_sample_count, np.sum(train_targets)/train_sample_count)
print(np.sum(validation_targets), validation_sample_count, np.sum(validation_targets)/validation_sample_count)
print(np.sum(test_targets), test_sample_count, np.sum(test_targets)/test_sample_count)

1794.0 3579 0.501257334450964
213.0 447 0.47651006711409394
230.0 448 0.5133928571428571


# Save the three datasets

In [11]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_input, targets=test_targets)

In [12]:
import tensorflow as tf

In [13]:
npz = np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs = npz['inputs'].astype(np.float)
validation_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs = npz['inputs'].astype(np.float)
test_targets = npz['targets'].astype(np.int)

# Building the Model

In [18]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(
    train_inputs,
    train_targets,
    batch_size=batch_size,
    epochs=max_epochs,
    callbacks=[early_stopping],
    validation_data=(validation_inputs, validation_targets),
    verbose=2
)

Epoch 1/100
36/36 - 0s - loss: 0.5073 - accuracy: 0.8145 - val_loss: 0.3741 - val_accuracy: 0.8971
Epoch 2/100
36/36 - 0s - loss: 0.3492 - accuracy: 0.8793 - val_loss: 0.2981 - val_accuracy: 0.8993
Epoch 3/100
36/36 - 0s - loss: 0.3109 - accuracy: 0.8852 - val_loss: 0.2787 - val_accuracy: 0.8949
Epoch 4/100
36/36 - 0s - loss: 0.2911 - accuracy: 0.8905 - val_loss: 0.2704 - val_accuracy: 0.9083
Epoch 5/100
36/36 - 0s - loss: 0.2790 - accuracy: 0.8952 - val_loss: 0.2664 - val_accuracy: 0.9038
Epoch 6/100
36/36 - 0s - loss: 0.2679 - accuracy: 0.8975 - val_loss: 0.2562 - val_accuracy: 0.9038
Epoch 7/100
36/36 - 0s - loss: 0.2622 - accuracy: 0.9008 - val_loss: 0.2622 - val_accuracy: 0.9105
Epoch 8/100
36/36 - 0s - loss: 0.2565 - accuracy: 0.9039 - val_loss: 0.2507 - val_accuracy: 0.9105
Epoch 9/100
36/36 - 0s - loss: 0.2506 - accuracy: 0.9030 - val_loss: 0.2545 - val_accuracy: 0.9083
Epoch 10/100
36/36 - 0s - loss: 0.2477 - accuracy: 0.9047 - val_loss: 0.2464 - val_accuracy: 0.9128
Epoch 11/

# Test the Model

In [19]:
test_loss, test_accuracy = model.evaluate(test_input, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.2854 - accuracy: 0.8906
